<a href="https://colab.research.google.com/github/sjdonado/covid19-xray-predictions/blob/master/covid19_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import shutil
import cv2

!pip install imutils

from imutils import paths

In [82]:
from google.colab import drive
drive.mount('/content/gdrive')
project_path = '/content/gdrive/My Drive/Kaggle/covid19-xray'
dataset_path = project_path + '/dataset'
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

%cd /content/gdrive/My Drive/Kaggle/covid19-xray

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle/covid19-xray


In [0]:
%%bash
rm -rf dataset
mkdir -p dataset/covid
mkdir -p dataset/normal
mkdir -p input/covid-chest-xray
mkdir -p input/chest-xray-pneumonia/chest_xray

In [84]:
# download covid dataset
covid_dataset_path = project_path + '/input/covid-chest-xray'
%cd /content/gdrive/My Drive/Kaggle/covid19-xray/input/covid-chest-xray

!kaggle datasets download -d bachrr/covid-chest-xray
!unzip -qq \*.zip
%rm *.zip

/content/gdrive/My Drive/Kaggle/covid19-xray/input/covid-chest-xray
 97% 204M/211M [00:02<00:00, 115MB/s]
100% 211M/211M [00:02<00:00, 97.3MB/s]


In [0]:
# construct the path to the metadata CSV file and load it
csvPath = os.path.sep.join([covid_dataset_path, "metadata.csv"])
df = pd.read_csv(csvPath)

# loop over the rows of the COVID-19 data frame
for (i, row) in df.iterrows():
    # if (1) the current case is not COVID-19 or (2) this is not
    # a 'PA' view, then ignore the row
    if row["finding"] != "COVID-19" or row["view"] != "PA":
        continue

    # build the path to the input image file
    imagePath = os.path.sep.join([covid_dataset_path, "images", row["filename"]])

    # if the input image file does not exist (there are some errors in
    # the COVID-19 metadeta file), ignore the row
    if not os.path.exists(imagePath):
        continue

    # extract the filename from the image path and then construct the
    # path to the copied image file
    filename = row["filename"].split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/covid", filename])

    # copy the image
    shutil.copy2(imagePath, outputPath)

In [86]:
# download pneumonia dataset
pneumonia_dataset_path = project_path + '/input/chest-xray-pneumonia/chest_xray'
%cd /content/gdrive/My Drive/Kaggle/covid19-xray/input/chest-xray-pneumonia/chest_xray

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -qq \*.zip
%rm *.zip

/content/gdrive/My Drive/Kaggle/covid19-xray/input/chest-xray-pneumonia/chest_xray
100% 2.29G/2.29G [00:25<00:00, 118MB/s]
100% 2.29G/2.29G [00:25<00:00, 95.9MB/s]


In [0]:
# build normal xray dataset
samples = 25
basePath = os.path.sep.join([pneumonia_dataset_path, "train", "NORMAL"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(42)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the filename from the image path and then construct the
    # path to the copied image file
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/normal", filename])

    # copy the image
    shutil.copy2(imagePath, outputPath)